# SAHI Benchmark

## 1. FULL INFERENCE

In [2]:
!nvidia-smi

Thu Oct 20 08:45:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Graphics...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P3     5W /  N/A |   1535MiB /  3913MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# arrange an instance segmentation model for test
from sahi.utils.yolov5 import (
    download_yolov5s6_model,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.scripts.coco_error_analysis import analyse
from sahi.scripts.coco_evaluation import evaluate
from IPython.display import Image
from pathlib import Path
import json
import os
from pathlib import Path
from PIL import Image

In [4]:
source_image_dir = "../resources/FLL_VAL/images/"
source_label_dir = "../resources/FLL_VAL/labels/"

coco_m_path = '../resources/models/yolov5m.pt'
fll_model_221007_path = '../resources/models/221007/best.pt'
fll_model_221012_path = '../resources/models/221012/best.pt'
fll_model_221013_path = '../resources/models/221013/best.pt'
fll_model_221014_path = '../resources/models/221014/best.pt'

In [5]:
fll_model_221012 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221012_path,
    confidence_threshold=0.25,
    device="cuda:0"
)

fll_model_221013 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221013_path,
    confidence_threshold=0.25,
    device="cuda:0"
)

fll_model_221014 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221014_path,
    confidence_threshold=0.25,
    device="cuda:0"
)

In [6]:
model = fll_model_221014
model_path = fll_model_221014_path

In [7]:
def initial_extract(img_dir, label_dir, out_dir):
    if os.path.exists(os.path.join(out_dir, 'val.json')):
        os.remove(os.path.join(out_dir, 'val.json'))
    
    licenses = [
        {
            "name": "",
            "id": 0,
            "url": ""
        }
    ]

    info_ = [
        {
            "contributor": "",
            "date_created": "",
            "description": "",
            "url": "",
            "version": "",
            "year": ""
        }
    ]

    categories = [
        {
            "id": 0,
            "name": "Buoy",
            "supercategory": ""
        },
        {
            "id": 1,
            "name": "Boat",
            "supercategory": ""
        },
        {
            "id": 2,
            "name": "Channel Marker",
            "supercategory": ""
        },
        {
            "id": 3,
            "name": "Speed Warning Sign",
            "supercategory": ""
        }
    ]

    img_idx = 0
    annot_idx = 0

    imgs_list = []
    annots_list = []

    for label_file in sorted(os.listdir(label_dir)):
        label_file_ = os.path.join(label_dir, label_file)
        img_file_ = os.path.join(img_dir, f'{os.path.splitext(label_file)[0]}.jpg')
        img = Image.open(img_file_)
        image_w, image_h = img.size

        imgs_list.append({
            'id': img_idx,
            'width': image_w,
            'height': image_h,
            'file_name': f'{os.path.splitext(label_file)[0]}.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        with open(label_file_, 'r') as label_f:
            labels = label_f.readlines()

            for label in labels:
                cat, xc, yc, label_normalized_w, label_normalized_h = list(map(lambda x: int(x) if len(x) == 1 else float(x), label.split()))
                label_w, label_h = image_w * label_normalized_w, image_h * label_normalized_h
                xmin, ymin = (image_w * xc) - (label_w / 2), (image_h * yc) - (label_h / 2)
                
                xmin = 0 if xmin < 0 else xmin
                ymin = 0 if ymin < 0 else ymin

                annots_list.append({
                    'id': annot_idx,
                    'image_id': img_idx,
                    'category_id': cat,
                    'area': int(label_h * label_w),
                    'bbox': [
                        xmin,
                        ymin,
                        label_w,
                        label_h
                    ],
                    'iscrowd': 0,
                    'attributes': {
                        'type': '',
                        'occluded': False
                    },
                    'segmentation': []
                })

                annot_idx += 1

        img_idx += 1

    out_dict = {
        'licenses': licenses,
        'info': info_,
        'categories': categories,
        'images': imgs_list,
        'annotations': annots_list
    }
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    
    with open(os.path.join(out_dir, 'val.json'), 'w') as out_f:
        print(os.path.join(out_dir, 'val.json'))
        json.dump(out_dict, out_f)
        
    return os.path.join(out_dir, 'val.json')

In [8]:
# initial_extract(img_dir, label_dir, out_dir)
gt_json_path = initial_extract(source_image_dir, source_label_dir, str(Path(source_image_dir).parent))

../resources/FLL_VAL/val.json


In [9]:
MODEL_TYPE = "yolov5"
MODEL_PATH = model_path
MODEL_CONFIG_PATH = ""
EVAL_IMAGES_FOLDER_DIR = source_image_dir
EVAL_DATASET_JSON_PATH = gt_json_path
INFERENCE_SETTING = "AVIKUS_FL"
EXPORT_VISUAL = False
MAX_DETECTIONS = 300

In [10]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "no_standard_prediction": False,
        "no_sliced_prediction": True,
        "slice_size": 512,
        "overlap_ratio": 0.15,
        "match_threshold": 0.5,
        "postprocess_class_agnostic": False,
        "custom_slice_y_start": 200,
    },
}

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

In [ ]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.25,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mode0",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    custom_slice_mode=0,
    custom_slice_x_start=0,
    custom_slice_y_start=0,
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

indexing coco dataset annotations...


Performing inference on images:   0%|                      | 5/5906 [00:00<04:39, 21.15it/s]

Prediction time is: 37.25 ms
Prediction time is: 31.34 ms
Prediction time is: 31.03 ms
Prediction time is: 34.76 ms
Prediction time is: 31.60 ms


Performing inference on images:   0%|                      | 8/5906 [00:00<04:19, 22.74it/s]

Prediction time is: 28.77 ms
Prediction time is: 30.01 ms
Prediction time is: 28.96 ms
Prediction time is: 33.96 ms
Prediction time is: 28.01 ms


...

Performing inference on images:  11%|██▎                 | 668/5906 [00:25<03:04, 28.42it/s]

Prediction time is: 25.45 ms
Prediction time is: 25.69 ms
Prediction time is: 26.05 ms
Prediction time is: 25.81 ms
Prediction time is: 25.60 ms
Prediction time is: 25.46 ms


Performing inference on images:  11%|██▎                 | 674/5906 [00:25<03:03, 28.50it/s]

Prediction time is: 25.57 ms
Prediction time is: 25.54 ms
Prediction time is: 24.99 ms
Prediction time is: 25.32 ms
Prediction time is: 25.05 ms
Prediction time is: 26.33 ms


Performing inference on images:  11%|██▎                 | 677/5906 [00:25<03:06, 27.98it/s]

In [ ]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

## Mode 2 (One Box)

In [13]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 512,
        "overlap_ratio": 0.15,
        "match_threshold": 0.5,
        "postprocess_class_agnostic": False,
        "custom_slice_x_start": 640,
        "custom_slice_y_start": 360,
        "custom_slice_mode": 2
    },
}

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

In [14]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.25,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mode0",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    custom_slice_mode=setting_params["custom_slice_mode"],
    custom_slice_x_start=setting_params["custom_slice_x_start"],
    custom_slice_y_start=setting_params["custom_slice_y_start"],
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

indexing coco dataset annotations...


Performing inference on images:   0%|                      | 2/5906 [00:00<07:14, 13.59it/s]

Performing prediction on 1 number of slices.
Prediction time is: 65.68 ms
Performing prediction on 1 number of slices.
Prediction time is: 58.41 ms
Performing prediction on 1 number of slices.


Performing inference on images:   0%|                      | 4/5906 [00:00<06:57, 14.14it/s]

Prediction time is: 56.89 ms
Performing prediction on 1 number of slices.
Prediction time is: 57.49 ms
Performing prediction on 1 number of slices.
Prediction time is: 65.29 ms
Performing prediction on 1 number of slices.


...

Performing prediction on 1 number of slices.
Prediction time is: 77.30 ms
Performing prediction on 1 number of slices.
Prediction time is: 77.96 ms
Performing prediction on 1 number of slices.


Performing inference on images: 100%|██████████████████▉| 5904/5906 [07:46<00:00, 11.43it/s]

Prediction time is: 68.74 ms
Performing prediction on 1 number of slices.
Prediction time is: 65.74 ms
Performing prediction on 1 number of slices.
Prediction time is: 81.68 ms
Performing prediction on 1 number of slices.


Performing inference on images: 100%|███████████████████| 5906/5906 [07:46<00:00, 12.66it/s]


Prediction time is: 82.18 ms
Performing prediction on 1 number of slices.
Prediction time is: 79.92 ms
Prediction results are successfully exported to runs/mode0/AVIKUS_FL8
Model loaded in 0.07174110412597656 seconds.
Slicing performed in 9.937860250473022 seconds.
Prediction performed in 398.3717291355133 seconds.


In [15]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.72s).
Accumulating evaluation results...
DONE (t=0.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=300 ] = 0.308
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=300 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=300 ] = 0.209
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=300 ] = 0.517
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=300 ] = 0.573
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=300 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=300 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.25,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="NMM",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mode0",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    custom_slice_mode=setting_params["custom_slice_mode"],
    custom_slice_x_start=setting_params["custom_slice_x_start"],
    custom_slice_y_start=setting_params["custom_slice_y_start"],
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

indexing coco dataset annotations...


Performing inference on images:   0%|                      | 2/5906 [00:00<08:29, 11.59it/s]

Performing prediction on 1 number of slices.
Prediction time is: 71.15 ms
Performing prediction on 1 number of slices.
Prediction time is: 73.43 ms
Performing prediction on 1 number of slices.


Performing inference on images:   0%|                      | 4/5906 [00:00<08:23, 11.71it/s]

Prediction time is: 65.77 ms
Performing prediction on 1 number of slices.
Prediction time is: 71.59 ms
Performing prediction on 1 number of slices.
Prediction time is: 65.47 ms
Performing prediction on 1 number of slices.


...

Prediction time is: 63.58 ms
Performing prediction on 1 number of slices.
Prediction time is: 73.45 ms
Performing prediction on 1 number of slices.
Prediction time is: 81.92 ms


Performing inference on images: 100%|██████████████████▉| 5903/5906 [07:26<00:00, 11.40it/s]

Performing prediction on 1 number of slices.
Prediction time is: 73.36 ms
Performing prediction on 1 number of slices.
Prediction time is: 69.90 ms
Performing prediction on 1 number of slices.


Performing inference on images: 100%|███████████████████| 5906/5906 [07:26<00:00, 13.22it/s]


Prediction time is: 81.44 ms
Performing prediction on 1 number of slices.
Prediction time is: 74.49 ms
Performing prediction on 1 number of slices.
Prediction time is: 69.89 ms
Prediction results are successfully exported to runs/mode0/AVIKUS_FL7
Model loaded in 0.05880093574523926 seconds.
Slicing performed in 10.395813703536987 seconds.
Prediction performed in 368.1465094089508 seconds.


In [12]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.05s).
Accumulating evaluation results...
DONE (t=0.44s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=300 ] = 0.187
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=300 ] = 0.038
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=300 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=300 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=300 ] = 0.416
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=300 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=300 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

## Mode 3 (Four Box)

In [ ]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 360,
        "overlap_ratio": 0.15,
        "match_threshold": 0.5,
        "postprocess_class_agnostic": False,
        "custom_slice_x_start": 580,
        "custom_slice_y_start": 280,
        "custom_slice_mode": 3
    },
}

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

In [ ]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.001,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mode3",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    custom_slice_mode=setting_params["custom_slice_mode"],
    custom_slice_x_start=setting_params["custom_slice_x_start"],
    custom_slice_y_start=setting_params["custom_slice_y_start"],
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

In [ ]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

## DSL

In [ ]:
!pwd

In [ ]:
dsl_result_json = "../../aiboat/aiboat/APP/NAS/deepstream-services-library/examples/python/result_standard_pred.json"

In [ ]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=dsl_result_json,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

## 2. SLICE INFERENCE

### Slice 좌표 찾기

In [ ]:
source_image_dir = "../resources/FLL_VAL/images/"
image_files = sorted([fn for fn in os.listdir(source_image_dir) if fn.endswith("jpg")])

In [ ]:
from typing import List
import copy
import matplotlib.pyplot as plt
import cv2
from ipywidgets import interact
%matplotlib inline 
from sahi import slicing
from sahi.slicing import slice_image

slicing.logger.setLevel(slicing.logging.INFO)

# single_row_y_start: int = 200,
@interact(index=(0, len(image_files)-1), slice_size=(0, 720), overlap_ratio=(0, 0.5, 0.05), single_row_y_start=(0, 512))
def visualize_slice_rect(index=0, slice_size=512, overlap_ratio=0.2, single_row_y_start=200):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    
    res = slice_image(image_path, 
                      slice_width=slice_size,
                      slice_height=slice_size,
                      overlap_height_ratio=overlap_ratio,
                      overlap_width_ratio=overlap_ratio,
                      single_row_y_start=single_row_y_start,
                      single_row_predict=True,
                      verbose=1)

    image = cv2.imread(image_path)
    image = copy.deepcopy(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    for start_pixel in res.starting_pixels:
        print(start_pixel)
        cv2.rectangle(image,
                      start_pixel,
                      [s1+s2 for s1, s2 in zip(start_pixel,[slice_size,slice_size])],
                      color=(255, 0, 0),
                      thickness=2)

    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

### SAHI

In [ ]:
MODEL_TYPE = "yolov5"
MODEL_PATH = model_path
MODEL_CONFIG_PATH = ""
EVAL_IMAGES_FOLDER_DIR = source_image_dir
EVAL_DATASET_JSON_PATH = gt_json_path
INFERENCE_SETTING = "AVIKUS_FL"
EXPORT_VISUAL = False
MAX_DETECTIONS = 500

In [ ]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 640,
        "overlap_ratio": 0.25,
        "match_threshold": 0.5,
        "postprocess_class_agnostic": False,
        "single_row_y_start": 200,
    },
}

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

In [ ]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.4,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
#     image_size=,
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
#     postprocess_type="NMS",
#     postprocess_match_metric="IOU",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mAP_TEST",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    single_row_predict=True,
    single_row_y_start=setting_params["single_row_y_start"]
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

In [ ]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

### DSL

In [ ]:
# network-input-shape= 5;3;640;640
# roi-params-src-0=0;0;1920;1080;0;200;640;640;480;200;640;640;960;200;640;640;1280;200;640;640

In [ ]:
dsl_result_json = "../../AiBoat/aiboat/APP/NAS/deepstream-services-library/examples/python/result_slice_pred.json"

In [ ]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=dsl_result_json,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)